In [ ]:
import logging
import os

import pandas as pd

import core.config.config_ as cconconf
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import helpers.hs3 as hs3
import helpers.hparquet as hparque
import helpers.hpandas as hpandas
import helpers.hdatetime as hdateti
import core.statistics as cstats
import im_v2.ccxt.data.client as icdcl
import im_v2.common.universe.universe as imvccunun
import research_amp.cc.statistics as ramccsta

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

# Configs

In [ ]:
def get_cmtask1680_config_ccxt() -> cconconf.Config:
    """
    Get task232-specific config.
    """
    config = cconconf.Config()
    # Load parameters.
    config.add_subconfig("load")
    config["load"]["aws_profile"] = "ck"
    config["load"]["data_dir"] = os.path.join("s3://cryptokaizen-data", "daily_staged")
    # Data parameters.
    config.add_subconfig("data")
    config["data"]["vendor"] = "CCXT"
    # Column names.
    config.add_subconfig("column_names")
    config["column_names"]["close_price"] = "close"
    config["column_names"]["currency_pair"] = "currency_pair"
    config["column_names"]["exchange_id"] = "exchange_id"
    return config

In [ ]:
config = get_cmtask1680_config_ccxt()
print(config)

# Functions

In [ ]:
def stat(currency_pair_list: list) -> pd.DataFrame:
    res = {}
    for currency_pair in currency_pair_list:
        data_loc = data.loc[data.currency_pair == currency_pair]
        days_availiable = (data_loc.last_valid_index() - data_loc.first_valid_index()).days
        n_data_points = data_loc.close.count()
        coverage =  100 * (1 - cstats.compute_frac_nan(data_loc.close))       
        res.update({currency_pair: [data_loc.index.min(), 
                                    data_loc.index.max(), 
                                    n_data_points,
                                    coverage,
                                    days_availiable,
                                    n_data_points / days_availiable,
                                   ]})
    return pd.DataFrame(
           data=res.values(),
           columns=["min_ts",
                     "max_ts",
                     "n_data_points",
                     "coverage", 
                     "days_available", 
                     "avg_data_points_per_day"],
           index=res.keys())

In [ ]:
def load_exchange_df(paths: list) -> pd.DataFrame:
    df = pd.DataFrame()
    for currency_pair, path in paths:
        data =  hpandas.read_csv_to_df(path)
        data['currency_pair'] = currency_pair
        df = pd.concat([df, data])
    return df

In [ ]:
def set_datetime_index(timestamps):
    times = []
    for time in timestamps:
        times.append(hdateti.convert_unix_epoch_to_timestamp(time))
    return times

# Load CCXT at cryptokaizen-data/daily_staged

In [ ]:
file_path = "s3://cryptokaizen-data/daily_staged/ccxt/binance/"
kwargs = {"aws_profile": "ck"}
data = hparque.from_parquet(file_path, **kwargs)
data.head()

## Calculate stats

In [ ]:
currency_pairs = list(data.currency_pair.unique())

In [ ]:
dfb = stat(currency_pairs)
dfb['exchange_id'] = "binance"
dfb['vendor'] = config["data"]["vendor"]
dfb

# Load CCXT at cryptokaizen-data/historical/

## binance stat

In [ ]:
file_path = "s3://cryptokaizen-data/historical/ccxt/latest/binance/"
kwargs = {"aws_profile": "ck"}
data = hparque.from_parquet(file_path, **kwargs)
data.head()

In [ ]:
currency_pairs = list(data.currency_pair.unique())
dfb = stat(currency_pairs)
dfb['exchange_id'] = "binance"
dfb['vendor'] = config["data"]["vendor"]
dfb

## bitfinex stat

In [ ]:
file_path = "s3://cryptokaizen-data/historical/ccxt/latest/bitfinex/"
kwargs = {"aws_profile": "ck"}
data = hparque.from_parquet(file_path, **kwargs)
data.head()

In [ ]:
currency_pairs = list(data.currency_pair.unique())
dfb = stat(currency_pairs)
dfb['exchange_id'] = "bitfinex"
dfb['vendor'] = config["data"]["vendor"]
dfb

## ftx stat

In [ ]:
file_path = "s3://cryptokaizen-data/historical/ccxt/latest/ftx/"
kwargs = {"aws_profile": "ck"}
data = hparque.from_parquet(file_path, **kwargs)
data.head()

In [ ]:
currency_pairs = list(data.currency_pair.unique())
dfb = stat(currency_pairs)
dfb['exchange_id'] = "ftx"
dfb['vendor'] = config["data"]["vendor"]
dfb

## gateio stat

In [ ]:
file_path = "s3://cryptokaizen-data/historical/ccxt/latest/gateio/"
kwargs = {"aws_profile": "ck"}
data = hparque.from_parquet(file_path, **kwargs)
data.head()

In [ ]:
currency_pairs = list(data.currency_pair.unique())
dfb = stat(currency_pairs)
dfb['exchange_id'] = "gateio"
dfb['vendor'] = config["data"]["vendor"]
dfb

## kucoin stat

In [ ]:
file_path = "s3://cryptokaizen-data/historical/ccxt/latest/kucoin/"
kwargs = {"aws_profile": "ck"}
data = hparque.from_parquet(file_path, **kwargs)
data.head()

In [ ]:
currency_pairs = list(data.currency_pair.unique())
dfb = stat(currency_pairs)
dfb['exchange_id'] = "kucoin"
dfb['vendor'] = config["data"]["vendor"]
dfb

# Load CCXT at cryptokaizen-data2/historical/

## binance stat

In [ ]:
paths = [("ADA_USDT", "s3://cryptokaizen-data2/historical/binance/ADA_USDT_20220210-104334.csv"),
         ("AVAX_USDT", "s3://cryptokaizen-data2/historical/binance/AVAX_USDT_20220210-105623.csv"),
         ("BNB_USDT", "s3://cryptokaizen-data2/historical/binance/BNB_USDT_20220210-110910.csv"),
         ("BTC_USDT", "s3://cryptokaizen-data2/historical/binance/BTC_USDT_20220210-112208.csv"),
         ("DOGE_USDT", "s3://cryptokaizen-data2/historical/binance/DOGE_USDT_20220210-113502.csv"),
         ("EOS_USDT", "s3://cryptokaizen-data2/historical/binance/EOS_USDT_20220210-114748.csv"),
         ("ETH_USDT", "s3://cryptokaizen-data2/historical/binance/ETH_USDT_20220210-120031.csv"),
         ("LINK_USDT", "s3://cryptokaizen-data2/historical/binance/LINK_USDT_20220210-121311.csv"),
         ("SOL_USDT", "s3://cryptokaizen-data2/historical/binance/SOL_USDT_20220210-122551.csv")]
data = load_exchange_df(paths)
data.head()

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.max())

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.min())

## bitfinex stat

In [ ]:
paths = [("ADA_USDT", "s3://cryptokaizen-data2/historical/bitfinex/ADA_USDT_20220211-161045.csv"),
         ("AVAX_USDT", "s3://cryptokaizen-data2/historical/bitfinex/AVAX_USDT_20220211-161212.csv"),
         ("BTC_USDT", "s3://cryptokaizen-data2/historical/bitfinex/BTC_USDT_20220211-161338.csv"),
         ("DOGE_USDT", "s3://cryptokaizen-data2/historical/bitfinex/DOGE_USDT_20220211-161507.csv"),
         ("EOS_USDT", "s3://cryptokaizen-data2/historical/bitfinex/EOS_USDT_20220211-161634.csv"),
         ("ETH_USDT", "s3://cryptokaizen-data2/historical/bitfinex/ETH_USDT_20220211-161801.csv"),
         ("FIL_USDT", "s3://cryptokaizen-data2/historical/bitfinex/FIL_USDT_20220211-161926.csv"),
         ("LINK_USDT", "s3://cryptokaizen-data2/historical/bitfinex/LINK_USDT_20220211-162053.csv"),
         ("SOL_USDT", "s3://cryptokaizen-data2/historical/bitfinex/SOL_USDT_20220211-162219.csv"),
         ("XRP_USDT", "s3://cryptokaizen-data2/historical/bitfinex/XRP_USDT_20220211-162345.csv"),]
data = load_exchange_df(paths)
data.head()

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.max())

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.min())

## ftx stat

In [ ]:
paths = [("BNB_USDT", "s3://cryptokaizen-data2/historical/ftx/BNB_USDT_20220210-104642.csv"),
         ("BNB_USDT", "s3://cryptokaizen-data2/historical/ftx/BNB_USDT_20220210-123958.csv"),
         ("BTC_USDT", "s3://cryptokaizen-data2/historical/ftx/BTC_USDT_20220210-110047.csv"),
         ("BTC_USDT", "s3://cryptokaizen-data2/historical/ftx/BTC_USDT_20220210-125404.csv"),
         ("DOGE_USDT", "s3://cryptokaizen-data2/historical/ftx/DOGE_USDT_20220210-111452.csv"),
         ("ETH_USDT", "s3://cryptokaizen-data2/historical/ftx/ETH_USDT_20220210-112851.csv"),
         ("LINK_USDT", "s3://cryptokaizen-data2/historical/ftx/LINK_USDT_20220210-114240.csv"),
         ("SOL_USDT", "s3://cryptokaizen-data2/historical/ftx/SOL_USDT_20220210-115701.csv"),
         ("XRP_USDT", "s3://cryptokaizen-data2/historical/ftx/XRP_USDT_20220210-121122.csv"),]
data = load_exchange_df(paths)
data.head()

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.max())

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.min())

## gateio stat

In [ ]:
paths = [("BNB_USDT", "s3://cryptokaizen-data2/historical/gateio/ADA_USDT_20220210-112115.csv"),
         ("AVAX_USDT", "s3://cryptokaizen-data2/historical/gateio/AVAX_USDT_20220210-113306.csv"),
         ("BNB_USDT", "s3://cryptokaizen-data2/historical/gateio/BNB_USDT_20220210-114500.csv"),
         ("BTC_USDT", "s3://cryptokaizen-data2/historical/gateio/BTC_USDT_20220210-115659.csv"),
         ("DOGE_USDT", "s3://cryptokaizen-data2/historical/gateio/DOGE_USDT_20220210-120851.csv"),
         ("EOS_USDT", "s3://cryptokaizen-data2/historical/gateio/EOS_USDT_20220210-122048.csv"),
         ("ETH_USDT", "s3://cryptokaizen-data2/historical/gateio/ETH_USDT_20220210-123244.csv"),
         ("FIL_USDT", "s3://cryptokaizen-data2/historical/gateio/FIL_USDT_20220210-124438.csv"),
         ("LINK_USDT", "s3://cryptokaizen-data2/historical/gateio/LINK_USDT_20220210-125629.csv"),
         ("SOL_USDT", "s3://cryptokaizen-data2/historical/gateio/SOL_USDT_20220210-130821.csv"),
         ("XRP_USDT", "s3://cryptokaizen-data2/historical/gateio/XRP_USDT_20220210-132013.csv"),]
data = load_exchange_df(paths)
data.head()

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.max())

In [ ]:
hdateti.convert_unix_epoch_to_timestamp(data.timestamp.min())